# **[ Project 03. Time-Series Forecasting: Predicting Stock Prices with ARIMA Model ]**



## **0. 문제정의**
---

### **0-1. 프로젝트 목표**  
<div>

</br>
본 프로젝트의 목표는 시계열 예측(Time-Series Prediction)을 다루는 여러 가지 통계적 기법 중에 가장 널리 알려진 ARIMA(Auto-regressive Integrated Moving Average)를 활용하여 특정 주식 종목의 가격을 예측해 보는것이다.
</div>




### **0.2. 분석 데이터 정의** 
<div> 
본 데이터는 
<a href = "https://finance.yahoo.com"> Yahoo Finance </a> 에서 제공하는 삼성,카카오,애플의 1년간(210505~220505)의 주가가 기록되어있다. 
</br>
데이터 아래 표와 같이 구성되어있다. 
</br>
</br>

| 컬럼 | 설명 |
| --- | --- |
| Date | 날짜 |
| Open | 시작가  |
| High | 최고가  |
| Low | 최저가 |
| Close | 종가 |
| Adj Close | 수정종가 |
| Volume | 거래량 |

</br>
본 프로젝트에서는 종가(Close)만 사용하도록하겠다. (주식 시장은 정해진 시간 동안만 거래가 가능한데, 종가란 하루의 장이 마감하였을 때의 가격을 뜻한다.)
</div> 



### **0.3 모델링 task 정의**  
<div> 
현실적으로 과거 주가데이터를 가지고 미래를 예측을 한다는 것은 불가능하다. 그럼에도 불구하고 미래를 예측하려한다면 적어도 <strong>데이터는 안정적(Stationary)</strong> 이라는 전제를 충족해야한다. 
</br>따라서 <strong>1) 데이터의 시계열 안정성 분석을 하고, 2) 데이터를 안정적인 시계열 형태라는 전제를 충족시킨 이후 3) ARIMA 모델로 예측</strong>을 진행하는 순서로 프로젝트를 진행하도록 하겠다.
</div> 
시계열 데이터를 분석하기 위해서 가장 먼저 문자형과 숫자형으로 준비된 데이터프레임을 시간자료구조를 포함한 적절한 데이터프레임으로 변환하는 과정이 필요하다. 시계열 데이터프레임이 준비되면 시각화를 통해 시계열 데이터 상태를 파악하고 정상 시계열 데이터로 변화하는 과정에 전념한다.



## **1. 데이터 불러오기 및 확인**
---

### **1-1. 라이브러리 불러오기**

In [17]:
############################################## 라이브러리 불러오기 #####################################################

import numpy as np
import pandas as pd
import platform
import random
import warnings
import statsmodels
import math

warnings.filterwarnings(action = "ignore")

# 시각화
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

if platform.system() == 'Darwin': #맥
    plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
    plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 코랩)    

    ! sudo apt-get update -qq
    ! sudo apt-get install fonts-nanum* -qq #나눔글꼴 설치
    ! sudo fc-cache -fv #폰트캐시 삭제

    # 나눔글꼴 matplotlib에 복사 (파이썬 버전 확인하는것 중요 !)
    ! sudo cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.7/dist-packages/matplotlib/mpl-data/fonts/ttf/
    # matplotlib 캐시삭제
    try:
      ! rm -rf /content/.cache/matplotlib/* #colab
    except:
      ! rm -rf /home/ubuntu/.cache/matplotlib/* #ubuntu


    sys_font=fm.findSystemFonts()
    nanum_font = [f for f in sys_font if 'NanumGothicCoding.ttf' in f][0]
    font_name = fm.FontProperties(fname=nanum_font, size=10).get_name()
    plt.rc('font', family=font_name)


plt.rcParams['axes.unicode_minus'] = False 
# fm._rebuild()

# 학습데이터분리, 교차검증
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV



# 시계열 관련 

from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.seasonal import seasonal_decompose  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf   
from statsmodels.tsa.arima_model import ARIMA   


# 모델 성능 평가 지표 
from sklearn.metrics import mean_squared_error, mean_absolute_error



/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 31 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


### **1-2. 데이터 불러오기**

In [24]:
############################## 데이터 로드 & 확인 ##############################

path = "https://raw.githubusercontent.com/saerannn/ML-DL-Project/main/Project_03_Time%20Series%20Forecasting%3A%20Predicting%20Stock%20Prices%20With%20ARIMA%20Model/data/"
samsung = pd.read_csv(path+"SamsungElectronics.csv", index_col='Date', parse_dates=True)
samsung = samsung['Close']
apple = pd.read_csv(path+'Apple.csv', index_col='Date', parse_dates=True)
apple = apple['Close']
kakao = pd.read_csv(path+'Kakao.csv', index_col='Date', parse_dates=True)
kakao = kakao['Close']

print("\n############################## samsung 데이터 확인 ##############################\n")
print(f' *데이터 크기 : {samsung.shape}')
print(f' *데이터 출력 : {samsung.head(5)}')

print("\n############################## apple 데이터 확인 ##############################\n")
print(f' *데이터 크기 : {apple.shape}')
print(f' *데이터 출력 : {apple.head(5)}')

print("\n############################## kakao 데이터 확인 ##############################\n")
print(f' *데이터 크기 : {kakao.shape}')
print(f' *데이터 출력 : {kakao.head(5)}')


############################## samsung 데이터 확인 ##############################

 *데이터 크기 : (246,)
 *데이터 출력 : Date
2021-05-06    82300.0
2021-05-07    81900.0
2021-05-10    83200.0
2021-05-11    81200.0
2021-05-12    80000.0
Name: Close, dtype: float64

############################## apple 데이터 확인 ##############################

 *데이터 크기 : (253,)
 *데이터 출력 : Date
2021-05-05    128.100006
2021-05-06    129.740005
2021-05-07    130.210007
2021-05-10    126.849998
2021-05-11    125.910004
Name: Close, dtype: float64

############################## kakao 데이터 확인 ##############################

 *데이터 크기 : (246,)
 *데이터 출력 : Date
2021-05-06    115000.0
2021-05-07    114500.0
2021-05-10    116000.0
2021-05-11    114500.0
2021-05-12    113000.0
Name: Close, dtype: float64


카카오와 삼성은 데이터가 246개이고, 애플은 253개인데 그 이유는 나라마다 주식거래 휴장일이 다르기 때문인것으로 예상된다.

## **2. 탐색적 데이터 분석 및 전처리**
---

### **2-1. 데이터 타입 확인**

In [33]:
############################## 컬럼 타입확인 ##############################



print("\n############################## samsung 데이터 타입확인 ##############################\n")
print(f' *Close컬럼 타입확인 : {samsung.dtypes}')

print("\n############################## apple 데이터 타입확인 ##############################\n")
print(f' *Close컬럼 타입확인 : {apple.dtypes}')

print("\n############################## kakao 데이터 타입확인 ##############################\n")
print(f' *Close컬럼 타입확인 : {kakao.dtypes}')



############################## samsung 데이터 타입확인 ##############################

 *Close컬럼 타입확인 : float64

############################## apple 데이터 타입확인 ##############################

 *Close컬럼 타입확인 : float64

############################## kakao 데이터 타입확인 ##############################

 *Close컬럼 타입확인 : float64


### **2-2. 통계값 확인**

In [42]:
############################## Close 통계값 확인 ##############################


print("\n############################## samsung 데이터 통계값확인 ##############################\n")
print(samsung.describe())

print("\n############################## apple 데이터 통계값확인 ##############################\n")
print(apple.describe())


print("\n############################## kakao 데이터 통계값확인 ##############################\n")
print(kakao.describe())







############################## samsung 데이터 통계값확인 ##############################

count      246.000000
mean     75079.674797
std       4594.192596
min      64800.000000
25%      70600.000000
50%      75500.000000
75%      79375.000000
max      83200.000000
Name: Close, dtype: float64

############################## apple 데이터 통계값확인 ##############################

count    253.000000
mean     153.822293
std       15.632815
min      122.769997
25%      144.979996
50%      151.119995
75%      166.559998
max      182.009995
Name: Close, dtype: float64

############################## kakao 데이터 통계값확인 ##############################

count       246.000000
mean     120451.626016
std       22422.662948
min       82600.000000
25%      100875.000000
50%      120000.000000
75%      137750.000000
max      169500.000000
Name: Close, dtype: float64


kakao는 최대값과 최솟값이 두배가량차이나는데 이게 이상치가 될 수 있수도있다. 우선 나중에 한번 살펴보기로한다.|

### **2-3. 결측값 확인**

In [43]:
############################## 결측여부 확인 ##############################



print("\n############################## samsung 데이터 결측여부 확인 ##############################\n")
print(samsung.isnull().any().any())

print("\n############################## apple 데이터 결측여부 확인 ##############################\n")
print(apple.isnull().any().any())


print("\n############################## kakao 데이터 결측여부 확인 ##############################\n")
print(kakao.isnull().any().any())




############################## samsung 데이터 결측여부 확인 ##############################

False

############################## apple 데이터 결측여부 확인 ##############################

False

############################## kakao 데이터 결측여부 확인 ##############################

False


### **2-4. 중복값 확인**

In [ ]:
############################## 중복된 항목 수 확인 ##############################
print("중복된 항목 수 :", len(digits_df[digits_df.duplicated()])) 

중복된 항목 수 : 0


### **2-5. 시계열 안정성 분석**

In [19]:
########### 시계열의 구성요소

# (1) 추세 요인 (Trend factor) 
# 인구의 변화, 자원의 변화, 자본재의 변화, 기술의 변화 등과 같은 요인들에 의해 영향을 받는 장기 변동 요인으로서, \
# 급격한 충격이 없는 한 지속되는 특성이 있습니다. "10년 주기의 세계경제 변동 추세" 같은 것이 추세 요인의 예라고 할 수 있습니다. 



# (2) 순환 요인 (Cycle factor) 
#  경제활동의 팽창과 위축과 같이 불규칙적이며 반복적인 중기 변동요인을 말합니다. 
# 주식투자가들이 "건설업/반도체업/조선업 순환주기"를 고려해서 투자한다고 말하는게 좋은 예입니다. 

# 만약 관측한 데이터셋이 10년 미만일 경우 추세 요인과 순환 요인을 구분하는 것이 매우 어렵습니다. 
# 그래서 관측기간이 길지 않을 경우 추세와 순환 요인을 구분하지 않고 그냥 묶어서 추세 요인이라고 분석하기도 합니다. 



# (3) 계절 요인 (Seasonal factor) 
#  12개월(1년)의 주기를 가지고 반복되는 변화를 말하며, 
# 계절의 변화, 공휴일의 반복, 추석 명절의 반복 등 과 같은 요인들에 의하여 발생합니다. 



# (4) 불규칙 요인 (Irregular / Random factor, Noise) 
#  일정한 규칙성을 인지할 수 없는 변동의 유형을 의미합니다.
#  천재지변, 전쟁, 질병 등과 같이 예 상할 수 없는 우연적 요인에 의해 발생되는 변동을 총칭합니다. 
# 불규칙변동 은 경제활동에 미미한 영향을 미치기도 하지만 때로는 경제생활에 지대한 영향을 주기도 합니다. 



# 출처: https://rfriend.tistory.com/509 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

In [ ]:
# 정상성 갖는 시계열 데이터 특징
# 시계열 Yt 모든 시점 t에 대해 평균이 μ로 같다.
# 시계열 Yt 모든 시점 t에 대해 분산이 σ2로 같다.
# 시계열 Yt, Ys 모든 h에 대해 공분산이 |t−s|=h로 같다; 예를 들어, 6단위 만큼 시점차이가 나는 경우 Cov(Y1,Y7)=Cov(Y11,Y17).


In [ ]:
# 안정성 판별법 : # Augmented Dickey-Fuller Test
# 귀무가설: 원계열은 안정적이지 않다
# p value 가 0.05 보다 작으면 귀무가설 기각 즉 안정적인 시계열
# 즉 결과값에 대한 pvalue는 정말 특별한 법칙을 따르는 것인지 ( p value 작음) 우연히 이런결과가 나왔는지( p value 큼) 의 척도이다


In [ ]:
# 불안정적인 시계열을 안정적인 시계열로 볒ㄴ경하는  방법
# 1) 정성적인 분석을 통해 보다 안정적(starionary)인 특성을 가지도록 기존의 시계열 데이터를 가공/변형하는 시도
    ## - 1. 시계열 평균이 비일정하면 원시계열에 차분(현 시점에서 전 시점의 자료 값을 뺌)하면 정상 시계열이 된다.
                  # diff(Yt)  을 통해 차분을 하계 되면 추세를 제거하는 효과를 거둠.
    ## - 2.계절차분: 계절성을 갖는 비정상시계열은 정상시계열로 바꿀 때 계절차분을 사용한다.
                  # diff(Yt) 을 통해 차분을 하계 되면 추세를 제거하는 효과를 거둠.

    ## - 3.로그변환: 분산이 일정하지 않은 경우 원계열에 자연로그(변환)을 취하면 정상시계열이 된다.
                  # (log(Yt) 을 통해 분산이 커지는 경향을 갖는 시계열을 안정화 시킴.)

​
# 2) 시계열 분해(Time series decomposition) 기법을 적용

#### **2-5-1. 정성적 그래프 분석**



In [ ]:
# 시계열(time series) 데이터를 차트로 그려 봅시다. 특별히 더 가공하지 않아도 잘 그려집니다.
plt.plot(ts1)

In [ ]:
########### 해석적기 
#  시간의 추이에 따라 시계열의 평균과 분산이 지속적으로 커지는 패턴을 보입니다.
#  rolling statistics를 추가해서 시각화해 보겠습니다.
# 이렇게 시간의 추이에 따라 평균과 분산이 증가하는 패턴을 보인다면 이 시계열 데이터는 적어도 안정적이진 않다고 정성적인 결론을 내려볼 수 있을 것 같습니다. 그렇다면 이런 시계열 데이터에 대해서는 시계열 예측을 시도할 수 없는 것일까요? 그렇다면 이번 노드는 너무 재미가 없겠지요?

# 이후 스텝들에는 이런 불안정적(Non-Stationary) 시계열 데이터에 대한 시계열 분석 기법을 다루어 볼 것입니다.

# 위와 같이 우리는 시계열 데이터의 안정성을 시각화 방법을 통해 정성적으로 분석해 보았습니다. 이것은 시계열 데이터를 다루는 가장 기본적인 접근법이라 할 수 있습니다.

# 하지만 시계열 데이터의 안정성을 평가하는 데는 보다 정량적인 방법이 있습니다. 다음 스텝에서 그 방법을 알아보도록 하겠습니다.



In [ ]:
############################## 구간 통계치(Rolling Statistics) 시각화 ##############################

# 구간 통계치(Rolling Statistics) 시각화하는 함수 정의
def plot_rolling_statistics(timeseries, window=12):
    
    rolmean = timeseries.rolling(window=window).mean()  # 이동평균 시계열
    rolstd = timeseries.rolling(window=window).std()    # 이동표준편차 시계열

     # 원본시계열, 이동평균, 이동표준편차를 plot으로 시각화해 본다.
    orig = plt.plot(timeseries, color='blue',label='Original')    
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

plot_rolling_statistics(ts1, window=12)

In [ ]:
########### 데이터 해석 적기 나중에 마크다운으로 변환해줘야한다 !!!!!!!!!!


# 해석 예시 
# 위의 그래프를 보면 시간의 추이에 따라 평균과 분산이 증가하는 패턴을 보이는데, 이는 불안정(Non-Stationary) 한 시계열 데이터라는 의미입니다.
# 이 시계열 데이터는 적어도 안정적이진 않다고 정성적인 결론을 내려볼 수 있을 것 같습니다. 
# 이 방법 외에 정량적으로 

In [ ]:
이렇게 시간의 추이에 따라 평균과 분산이 증가하는 패턴을 보인다면 이 시계열 데이터는 적어도 안정적이진 않다고 정성적인 결론을 내려볼 수 있을 것 같습니다.

하지만 시계열 데이터의 안정성을 평가하는 데는 보다 정량적인 방법이 있습니다. 다음 스텝에서 그 방법을 알아보도록 하겠습니다.



#### **2-5-2. 정량적 Augmented Dicky-Fuller Test**


In [ ]:
# 그럼 이전 스텝에서 정성적으로 분석해 보았던 두 시계열(Time Series)에 대한 
# Augmented Dickey-Fuller Test를 수행해 봅시다.

In [ ]:
# 주어진 시계열 데이터가 안정적이지 않다라는 귀무가설(Null Hypothesis)를 세운 후,

# 통계적 가설 검정 과정을 통해 이 귀무가설이 기각될 경우에

# 이 시계열 데이터가 안정적이다라는 대립가설(Alternative Hypothesis)을 채택한다



In [ ]:
def augmented_dickey_fuller_test(timeseries):
    # statsmodels 패키지에서 제공하는 adfuller 메서드를 호출합니다.
    dftest = adfuller(timeseries, autolag='AIC')  
    
    # adfuller 메서드가 리턴한 결과를 정리하여 출력합니다.
    print('Results of Dickey-Fuller Test:')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [ ]:
# augmented_dickey_fuller_test(ts1)

In [ ]:
# ts1(Daily Minimum Temperatures in Melbourne)시계열이 안정적이지 않다는 귀무가설은 p-value가 거의 0에 가깝게 나타났습니다.

# 따라서 이 귀무가설은 기각되고, 이 시계열은 안정적 시계열이라는 대립가설이 채택됩니다.

In [ ]:
# augmented_dickey_fuller_test(ts2)

In [ ]:
#############3

# ADF(Augmented Dickey-Fuller) 테스트를 통해 '현대'의 주가에 대한 안정성(stationary)을 확인한 결과 
# 유의수준 0.05에서 유의수준 값 0.437이므로 대립가설을 기각하고 귀무가설을 채택하여,
# '현대의 주가는 안정적이지 않다'고 할 수 있습니다.

In [ ]:
######### 정성 정량을 통한 해석
# 위의 그래프와 ADF Test 를 통해 불안정(Non-Stationary) 한 시계열 데이터라고 판단을 내렸다. 
# 로그 변환을 통해서 안정한(Stationary) 시계열 데이터로 바꾸도록 하겠습니다.

#### **2-5-3. 안정적인 시계열 형태로 변환**


정성적인 분석을 통해 보다 안정적(starionary)인 특성을 가지도록 기존의 시계열 데이터를 가공/변형하는 시도

시계열 분해(Time series decomposition) 기법을 적용

In [ ]:
#### **2-5-3. 시계열 분해(Time Series Decomposition)**

#### **2-5-4. Residual 안정성 확인**

### **2-6. 시계열 분해**


## **3. 모델 구현 및 평가**
---

#### **3-1. train/test 나누기**


In [ ]:
############################## 학습데이터 나누기 & 교차검증 ##############################

X = digits_data_scaled
y = digits_label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, randomstate=0)


### **3-2. 적정 ARIMA 모수 찾기**


#### **3-2-1. ACF, PACF 그려보기**


In [ ]:
############################## p,q 구하기 ##############################


#### **3-2-2. 차분 안정성 확인** 

In [ ]:
############################## p,q 구하기 ##############################

### **3-3. 모델 학습 및 예측**

### **3-4. 모델 성능 평가**